<a href="https://colab.research.google.com/github/nayse/mlearning/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><a href="https://www.nvidia.com/dli"> <img src="https://drive.google.com/uc?id=1J3bpGyJcz-7uOFkUNhvOBiReBCk-sUwR" width="200" </a></center>

# Assessment

In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started!

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive/DISCIPLINAS/AM/desafio6/fruits"

!ls

/content/gdrive/MyDrive/DISCIPLINAS/AM/desafio6/fruits
train  valid


## Load ImageNet Base Model

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

In [ ]:
#congelando as camadas do modelo base
base_model.trainable = False


## Add Layers to Model

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## Compile Model

In [ ]:
#compilando o modelo
#configura a função de perda e as métricas para treinamento em várias categorias

model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'])


## Augment the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#cria um gerador de dados de imagem com aumentação
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)


## Load Dataset

In [ ]:
# carregando dataset de teste

%cd "/content/gdrive/MyDrive/DISCIPLINAS/AM/desafio6/fruits"



train_it = datagen.flow_from_directory("/content/gdrive/MyDrive/DISCIPLINAS/AM/desafio6/fruits/train",
                                       target_size=(224, 224),
                                       color_mode='rgb',
                                       class_mode="categorical")
# carregando dataset de validação
valid_it = datagen.flow_from_directory("/content/gdrive/MyDrive/DISCIPLINAS/AM/desafio6/fruits/valid",
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical")

/content/gdrive/MyDrive/DISCIPLINAS/AM/desafio6/fruits
Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

Epoch 1/10
36/36 [==============================] - 450s 12s/step - loss: 2.2430 - accuracy: 0.5220 - val_loss: 1.1253 - val_accuracy: 0.6717
Epoch 2/10
36/36 [==============================] - 34s 914ms/step - loss: 0.7267 - accuracy: 0.7927 - val_loss: 0.6813 - val_accuracy: 0.8389
Epoch 3/10
36/36 [==============================] - 34s 920ms/step - loss: 0.3943 - accuracy: 0.8697 - val_loss: 0.5031 - val_accuracy: 0.8359
Epoch 4/10
36/36 [==============================] - 33s 905ms/step - loss: 0.2569 - accuracy: 0.9120 - val_loss: 0.3313 - val_accuracy: 0.8997
Epoch 5/10
36/36 [==============================] - 33s 901ms/step - loss: 0.1791 - accuracy: 0.9408 - val_loss: 0.2692 - val_accuracy: 0.9179
Epoch 6/10
36/36 [==============================] - 33s 895ms/step - loss: 0.1336 - accuracy: 0.9543 - val_loss: 0.2232 - val_accuracy: 0.9149
Epoch 7/10
36/36 [==============================] - 33s 890ms/step - loss: 0.1212 - accuracy: 0.9560 - val_loss: 0.1557 - val_accuracy: 0.9453


## Unfreeze Model for Fine Tuning

In [ ]:
#descongelar o modelo base
base_model.trainable = True

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


## Evaluate the Model

"You will need to get the model to a validation accuracy of 92% in order to pass the assessment."

In [ ]:
evaluation_result = model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)


if evaluation_result[1] >= 0.92:
    print("Congratulations! You passed the assessment!")

else:
    print("Unfortunately, you did not pass the assessment. Try again.")

10/10 [==============================] - 8s 792ms/step - loss: 0.2250 - accuracy: 0.9483
Congratulations! You passed the assessment!
